In [105]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

## function to load images and labels

In [106]:
def load_classes(classes_path):
    with open(classes_path, 'r') as file:
        classes = file.read().splitlines()
    return classes

def load_dataset(dataset_path, classes):
    images = []
    labels = []
    for folder_name in ['train', 'test']:  # removed 'validation'
        folder_path = os.path.join(dataset_path, folder_name)
        annotations_path = os.path.join(folder_path, '_annotations.txt')
        with open(annotations_path, 'r') as file:
            for line in file.readlines():
                parts = line.strip().split()
                image_path = os.path.join(folder_path, parts[0])
                bounding_boxes = parts[1:]
                image = load_img(image_path, target_size=(64, 64))
                image = img_to_array(image)
                images.append(image)
                labels_for_image = []
                for bbox in bounding_boxes:
                    x_min, y_min, x_max, y_max, class_id = map(int, bbox.split(','))
                    class_label = classes[class_id]  # convert class ID to class label
                    labels_for_image.append([x_min, y_min, x_max, y_max, class_label])
                labels.append(labels_for_image)
    images = np.array(images)
    return images, labels  # labels is a list of lists of bounding boxes

## function to preprocess images and labels

In [107]:
def preprocess(images, labels):
    images = images.astype('float32') / 255.0
    labels = to_categorical(labels)
    return images, labels

## define model 

In [108]:
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

## Load and preprocess the dataset

In [109]:
classes_path = r'C:\Users\suraj\OneDrive\Desktop\Projects\internship_applications_project\website_snapshot_element_identifier\Website Screenshots.v1-raw.yolov4pytorch\train\_classes.txt'
classes = load_classes(classes_path)
dataset_path = r'C:\Users\suraj\OneDrive\Desktop\Projects\internship_applications_project\website_snapshot_element_identifier\Website Screenshots.v1-raw.yolov4pytorch'
images, labels = load_dataset(dataset_path, classes)

## train test split

In [110]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

## create model

In [111]:
input_shape = X_train.shape[1:]
num_classes = len(set(class_label for sublist in y_train for _, _, _, _, class_label in sublist))
print(num_classes)

8


## compile model

In [112]:
from keras.models import Sequential
from keras.layers import Dense

input_dim = X_train.shape[1] * X_train.shape[2] * X_train.shape[3]  # calculate input_dim

# Flatten the images
X_train = X_train.reshape(X_train.shape[0], input_dim)
X_val = X_val.reshape(X_val.shape[0], input_dim)

model = Sequential()
model.add(Dense(32, input_shape=(input_dim,), activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## train model

In [113]:
#If your y_train and y_val are lists of lists where the last element is the class label, you can convert them to NumPy arrays of class labels 
import numpy as np

y_train = np.array([label[-1] for label in y_train])
y_val = np.array([label[-1] for label in y_val])

In [114]:
from sklearn.preprocessing import LabelEncoder
y_train_labels = y_train[:, -1]
y_val_labels = y_val[:, -1]

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train_labels)
y_val_encoded = encoder.transform(y_val_labels)

y_train = to_categorical(y_train_encoded, num_classes=8)
y_val = to_categorical(y_val_encoded, num_classes=8)

ValueError: y should be a 1d array, got an array of shape (1544, 5) instead.

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10


ValueError: in user code:

    File "c:\Users\suraj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\suraj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\suraj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\suraj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\suraj\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    F

## model evaluation

In [ ]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Loss:', loss)
print('Accuracy:', accuracy)